In [5]:
import math
import pandas as pd
import numpy as np
import random

# from sklearn.metrics import accuracy_score

#切分文本,得到数据集
def loadDataSet(file):
    df = pd.read_table(file)
    textList = df.Phrase.map(lambda x: x.lower()).str.split('\W+')
    classVec= df['Sentiment']
    return textList,classVec
  
textList, classVec = loadDataSet('train.tsv')
print(textList,classVec)


# if __name__ == "__main__":
#     textList, classVec = loadDataSet('train.tsv')
#     vocabList = createVocabList(textList)
#     returnVec = SetOfWords2Vec(vocabList, textList)
#     print(returnVec)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  # Remove the CWD from sys.path while we load stuff.


0         [a, series, of, escapades, demonstrating, the,...
1         [a, series, of, escapades, demonstrating, the,...
2                                               [a, series]
3                                                       [a]
4                                                  [series]
5         [of, escapades, demonstrating, the, adage, tha...
6                                                      [of]
7         [escapades, demonstrating, the, adage, that, w...
8                                               [escapades]
9         [demonstrating, the, adage, that, what, is, go...
10                              [demonstrating, the, adage]
11                                          [demonstrating]
12                                             [the, adage]
13                                                    [the]
14                                                  [adage]
15                  [that, what, is, good, for, the, goose]
16                                      

In [6]:
# 根据词频选择部分词汇作为vocabulary
def createVocabList(textList):
#     vocabSet0 = set([])
    vocabListSet = {}
    for document in textList:
#         vocabSet0 = vocabSet0 | set(document)
        for word in document:
            if word not in vocabListSet.keys():
                vocabListSet[word] = 1
            else:
                vocabListSet[word] += 1  
    sorted_vocabListSet = sorted(vocabListSet.items(),key=lambda x:x[1],reverse=True)
    stwlist = [line.strip() for line in open('stopwords.txt', encoding='utf-8').readlines()]
    sorted_vocabList = []
    for i in range(len(sorted_vocabListSet)):
        sorted_vocabList.append(sorted_vocabListSet[i][0])
#     return sorted_vocabList

    vocabList = []
    for word in sorted_vocabList:
        if word not in stwlist and vocabListSet[word] >800 and len(word)>1:
            vocabList.append(word)
#     return sorted_vocabListSet
    return vocabList

vocabList = createVocabList(textList)
print(vocabList)
print(len(vocabList))

['film', 'movie', 'story', 'rrb', 'good', 'lrb', 'time', 'characters', 'comedy', 'life', 'funny', 'make', 'movies', 'director', 'love', 'bad', 'work', 'made', 'action', 'people', 'makes', 'character', 'world', 'plot', 'films', 'drama', 'audience', 'long', 'big', 'sense', 'man', 'feel', 'real', 'great', 'humor', 'screen', 'year', 'minutes']
38


In [7]:
# one-hot编码
def SetOfWords2Vec(vocabList,textList):
    returnVec = []
    for i in  range(len(textList)):                
        returnVeci = [0]*len(vocabList)
        for word in textList.iloc[i]:
            if word in vocabList:
                returnVeci[vocabList.index(word)] = 1
        returnVec.append(returnVeci)
    return returnVec

returnVec = SetOfWords2Vec(vocabList,textList)
# print(returnVec[0:10])
print(np.array(returnVec).shape)

(156060, 38)


In [8]:
from sklearn.model_selection import train_test_split
#定义softmax函数
class Softmax(object):
    def __init__(self):
        self.learning_rate = 0.0001           # 学习速率
        self.max_iteration = 10000            # 最大迭代次数
        self.weight_lambda = 0.01             # 衰退权重

    def cal_e(self,x,l):
        '''
        softmax分子
        '''
        theta_l = self.w[l]
        product = np.dot(theta_l,x)
        return math.exp(product)

    def cal_probability(self,x,j):
        '''
        softmax
        '''
        molecule = self.cal_e(x,j)
        denominator = sum([self.cal_e(x,i) for i in range(self.k)])
        return molecule/denominator

    def cal_partial_derivative(self,x,y,j):
        '''
        计算theta j 的梯度
        '''
        first = int(y==j)                           # 计算示性函数
        second = self.cal_probability(x,j)          # 计算后面那个概率
        return -x*(first-second) + self.weight_lambda*self.w[j]
       
    def train(self, features, labels):
        self.k = len(set(labels))
        self.w = np.zeros((self.k,len(features[0])+1))  # w的维度：k*特征个数+1
        time = 0
        while time < self.max_iteration:
#             print('loop %d' % time)
            time += 1
            index = random.randint(0, len(labels) - 1) #产生一个随机数
            x = features[index]
            y = labels[index]
            
            x = list(x)
            x.append(1.0)
            x = np.array(x)

            derivatives = [self.cal_partial_derivative(x,y,j) for j in range(self.k)]
            for j in range(self.k):
                self.w[j] -= self.learning_rate * derivatives[j]  #梯度下降
        return self.w
        
    
        
if __name__=="__main__":   
    train_features, test_features, train_labels, test_labels = train_test_split(
        returnVec, classVec, test_size=0.3, random_state=2)
    w = Softmax().train(train_features , list(train_labels))
    print("w的维度：" + str(w.shape))
    print("test_features的维度：" + str(np.array(test_features).shape))
    print("test_labels的维度：" + str(np.array(test_labels).shape))

    
#     print(list(train_labels))


w的维度：(5, 39)
test_features的维度：(46818, 38)
test_labels的维度：(46818,)


In [0]:
def predict_(w,x):
        result = np.dot(x,np.transpose(w))
        row, column = result.shape
        _positon = np.argmax(result)  # 找最大值所在的列
        m, n = divmod(_positon, column)
        return m

def predict(features):
    labels = []
    for feature in features:
        x = list(feature)
        x.append(1)

        x = np.matrix(x)
        x = np.transpose(x)
        labels.append(predict_(x))
    return labels

def feature_plus(feature):
    feature_plus = []
    for i in range(len(feature)):
        feat = feature[i]
        feat.append(1.0)
        feature_plus.append(feat)
    return feature_plus

test_features_1 = feature_plus(test_features)
M = [] 
for i in range(len(test_features_1)):
    m = predict_(w,test_features_1)
    M.append(m)
return M
print(M)